# Demo: Data from a Bluesky Post


Let's see what the data actually looks like from a Bluesky Post!

First we need to do our normal Bluesky login steps (and optional fake atproto step)
## Log into atproto (or fake atproto)
### Load atproto library

In [1]:
# Load some code called "Client" from the "atproto" library that will help us work with Bluesky
from atproto import Client

### (Optional) Step 1b: Make a fake Bluesky connection with the fake_atproto library
For testing purposes, we"ve added this line of code, which loads a fake version of atproto, so it wont actually connect to Bluesky. __If you want to try to actually connect to Bluesky, don't run this line of code.__

In [2]:
%run ../../fake_apis/fake_atproto.ipynb

### Step 2: Login to Bluesky
To use this on your real Bluesky account, copy your [bluesky account name and login](../../appendix/making_bot_account.md) into the code below, replacing our fake bluesky name and password.

In [3]:
# Login to Bluesky
# TODO: put your account name and password below

client = Client(base_url="https://bsky.social")
client.login("your_account_name.bsky.social", "m#5@_fake_bsky_password_$%Ds")

## Find Bluesky data
Below I have the code to find a recent Bluesky post from the feed [Animal](https://bsky.app/profile/did:plc:2m44csq7rgph66s2kltbypxh/feed/aaab56iiatpdo) by the user [shouldhaveanimal.bsky.social](https://bsky.app/profile/shouldhaveanimal.bsky.social).

Don't worry if you don't understand this part yet. We are
just doing this, so we can get to the point of seeing what tweet data looks like.

_Note: If you run this on real Bluesky, we can't gurantee anything about how offensive what you might find is._

In [4]:
feed = client.app.bsky.feed.get_feed({'feed': 'at://did:plc:2m44csq7rgph66s2kltbypxh/app.bsky.feed.generator/aaab56iiatpdo'}).feed
recent_post = feed[0].post

## Look at data in Bluesky submission

Now we will look at some of the data that came back!

Again, don't worry too much about the code, we want to look at the data and data types.

### post text:

In [5]:
display("The data type of the post text is: " + type(recent_post.record.text).__name__)
display("The post text is: " + recent_post.record.text)

'The data type of the post text is: str'

'The post text is: Look at my cute dog!'

As you can see above, the text of a post is a string (`str`) data type. 

### post content id (cid)

In [6]:
display("The data type of the post content id is: " + type(recent_post.cid).__name__)
display("The post content id is: " + str(recent_post.cid))

'The data type of the post content id is: str'

'The post content id is: 904tjwdf093j'

The post content id is a piece of text (`str`) that looks like random letters and numbers. This is how the post is referred to inside Bluesky's computers. So if someone is commenting on a post, Bluesky just puts uses content ID they were commenting on to see where to display it.

### submission author handle

In [7]:
display("The data type of the author handle is: " + type(recent_post.author.handle).__name__)
display("The author handle is: " + str(recent_post.author.handle))

'The data type of the author handle is: str'

'The author handle is: fake_user.bsky.social'

The author name is an string (`str`). Note recent_post.author.author has other information about the author as well as the name.

### post created at

In [8]:
display("The data type of the post created_at is: " + type(recent_post.record.created_at).__name__)
display("The created_at at is: " + str(recent_post.record.created_at))

'The data type of the post created_at is: str'

'The created_at at is: 2014-01-01'

The created at time for the submission is a string (`str`), which is in [Coordinated Universal Time](https://en.wikipedia.org/wiki/Coordinated_Universal_Time) zone.

### number of likes (score)

In [9]:
display("The data type of the number of likes is: " + type(recent_post.like_count).__name__)
display("The number of likes is: " + str(recent_post.like_count))

'The data type of the number of likes is: int'

'The number of likes is: 23'

The number of likes is is a whole number (`int`). 

### number of replies

In [10]:
display("The data type of the number of replies is: " + type(recent_post.reply_count).__name__)
display("The number of replies is: " + str(recent_post.reply_count))

'The data type of the number of replies is: int'

'The number of replies is: 7'

The number of replies is is a whole number (`int`). Note: You can also get a data structure of all the comments, but we will look at that later. 

### number of reposts

In [11]:
display("The data type of the number of reposts is: " + type(recent_post.repost_count).__name__)
display("The number of reposts is: " + str(recent_post.repost_count))

'The data type of the number of reposts is: int'

'The number of reposts is: 5'

The number of reposts is a whole number (`int`). 

The submission url is a string (`str`).

## Still more!
In addition to the data we looked at above, there are even more options for bluesky posts. The documentation seems them seems a bit incomplete, but you can see [more info about posts here](https://atprotodart.com/docs/lexicons/app/bsky/feed/defs#postview): 